### Change directory to kaolin

In [3]:
%cd kaolin

/home/mmvc/mmvc-ad-teachnas/ML_2020_Spring/sms1198/ml_autodecoder_ensemble_net_for_3D_objects/kaolin


In [9]:
# import necessary libraries
from kaolin.datasets import modelnet


In [ ]:
voxels = modelnet.ModelNet(root='../data/__MACOSX/ModelNet10/toilet/', categories=['planes'])